In [55]:
faiss.__version__

'1.7.2'

In [ ]:
# pip install faiss-gpu 는 requirements.txt 에 포함된다면 가능

In [ ]:
pip install faiss-gpu

In [53]:
from ast import literal_eval
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import faiss
import random

warnings.filterwarnings(action='ignore')
books = pd.read_csv('/content/drive/MyDrive/nlbooks.csv', encoding='utf-8')
books = books.replace({np.nan: 'none'})

# 데이터프레임이 content, thumbnail column을 포함한다면
# books_df=books[['isbn13','title','author','publisher','price','img_url','description','kdc_class_no']]
books_df=books[['isbn13','title','author','publisher','price','img_url','description','kdc_class_no']]

#books_df[['description']=='none']=None
books_df_nax=books_df.dropna(axis='rows')


tfidf = TfidfVectorizer(stop_words='english')
# fit_transform안에 데이터프레임형태로 넣어주면 안됨. 하나의 변수씩만 넣어주자!
# title_vect1 = cnt_vect.fit_transform(books_df_nax['title'])
title_vect2 = tfidf.fit_transform(books_df_nax['title'])
#des_vect = cnt_vect.fit_transform(books['description'])
#books['book_info']=books['title']+books['description']
#b_category=books['kdc_class_no']

# 차원 축소 part 를 줄여야 함 -> 파일을 새로 저장해야할듯!
X = title_vect2
svd = TruncatedSVD(n_components=500, n_iter=7, random_state=42)
X_new=svd.fit_transform(X)

D = X_new.shape[1]
K = 500  # The number of clusters
X = X_new.astype(np.float32)

# Setup
kmeans = faiss.Kmeans(d=D, k=K, niter=20, verbose=True,gpu=True)
# For GPU(s), run the following line. This will use all GPUs
# kmeans = faiss.Kmeans(d=D, k=K, niter=20, verbose=True, gpu=True)

# Run clustering
kmeans.train(X)

# Error for each iteration
# print(kmeans.obj)  # array with 20 elements

# Centroids after clustering
# print(kmeans.centroids.shape)  # (10, 500)

# The assignment for each vector.
dists, ids = kmeans.index.search(X, 1)  # Need to run NN search again
books_df_nax['tit_cluster']=ids


def find_sim_books(U):
    def userbooks(A):
        l=len(A)
        n=random.randrange(0,l)
        return A[n]

    # 사용자의 isbn 데이터를 담은 책 리스트를 input으로 받아 그 리스트 중 하나를 반환하는 함수
    # B = 사용자 데이터에서 랜덤 추출한 책의 isbn 
    B=userbooks(U)


    def sim_idx_with_A(df,isbn):
        n=int(df[df['isbn13']==isbn]['tit_cluster'])
        same_clu_books_df=df[df['tit_cluster']==n]
        sen = same_clu_books_df['description']

        # 1번 후보 : sentence transformer 유사도 구하기
        #from sentence_transformers import SentenceTransformer
        #model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        #sen_embeddings = model.encode(sen)
    
        # 2번 후보 : 단순 tfidf 활용한 유사도 구하기
        clu_tfidf = tfidf.fit_transform(sen)
        clu_des_sim=cosine_similarity(clu_tfidf,clu_tfidf)

        # idx_list=same_clu_books_df.index[same_clu_books_df['title']==books].tolist()
        idx=np.where(same_clu_books_df['isbn13']==B)[0][0]
        books_sim_vect=clu_des_sim[idx:idx+1]
        books_des_sim_idx = books_sim_vect.argsort()[::-1]

        return books_des_sim_idx

    # 사용자 책 리스트에서 랜덤 추출한 책 기준 유사한 책 인덱스 리스트 반환
    sim_books_idx=sim_idx_with_A(books_df_nax,B)

    #books=userbooks(A)
    #n=int(df[df['title']==books]['tit_cluster'])
    #same_clu_books_df=df[df['tit_cluster']==n]
    #sen = same_clu_books_df['description']
    top_n=20
    top_sim_idx=sim_books_idx[0][:top_n]
    top_sim_idx=top_sim_idx.reshape(-1,)

    sim_books=books_df_nax.iloc[top_sim_idx]
    # 데이터프레임이 content, thumbnail column을 포함한다면
    # outputs=sim_books[['title','author','content','thumbnail','publisher','kdc','price']]
    outputs=sim_books[['title','author','img_url','publisher','kdc_class_no','price']]
    outputs_dic=outputs.to_dict('records')

    return outputs_dic

In [ ]:
# 출력 확인 예시

In [8]:
b=books_df_nax.to_dict('records')

In [9]:
b[0:3]

[{'isbn13': 9790000000000.0,
  'title': '너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이',
  'author': '이용신 (지은이)',
  'publisher': '푸른숲',
  'price': 16000.0,
  'description': '2004년 방영한 애니메이션 &lt;달빛천사&gt;에서 주인공 루나(풀문) 역을 맡으며 90년대생들에게 보석 같은 추억을 선물한 성우 이용신의 첫 번째 에세이. 수많은 작품의 주연을 맡으며 쉬지 않고 대중에게 행복을 전해온 성우 이용신의 발자취를 확인할 수 있다.',
  'kdc_class_no': 'none'},
 {'isbn13': 9790000000000.0,
  'title': '일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이',
  'author': '차도하 (지은이)',
  'publisher': '위즈덤하우스',
  'price': 15800.0,
  'description': '“그러니 나는 말하고 싶은 것을 말하겠다”「침착하게 사랑하기」 차도하 시인 첫 에세이새롭고 도발적인 작품성으로 문단의 기대주로 떠오른 차도하 시인의 첫 번째 산문집이 출간됐다. 혼자 보는 일기에도 거짓말을 쓸 수밖에 없었던 시인의 산문집 『일기에도 거짓말을 쓰는 사람』은 공교롭게도 보는...',
  'kdc_class_no': 'none'},
 {'isbn13': 9790000000000.0,
  'title': '본격 한중일 세계사 12 - 임오군란과 통킹 위기',
  'author': '굽시니스트 (지은이)',
  'publisher': '위즈덤하우스',
  'price': 14800.0,
  'description': '한중일 관계의 결정적 분기점인 임오군란의 막전 막후를 다룬다. 러시아의 세력 확장을 경계한 청은 ‘친중·결일·연미’라는 계책을 앞세워 조선에 대한 통제를 강화하고, 이를 그냥 보아 넘길 수 없던 일본은 임오군란을 계기로 마수를 